In [36]:
import googlemaps
from datetime import datetime
import os
import requests
import json
import time
import pandas as pd
import folium
from dotenv import load_dotenv
load_dotenv()

True

In [39]:
def searchPlaces(query, location, radius):
    API_key = os.getenv('key')
    endpoint_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    places = []
    params = {
        'query': query,
        'location': location,
        'radius': radius,
        'key': API_key
    }
    res = requests.get(endpoint_url, params = params)
    results =  json.loads(res.content)
    places.extend(results['results'])
    time.sleep(2)
    while "next_page_token" in results:
        params['pagetoken'] = results['next_page_token'],
        res = requests.get(endpoint_url, params = params)
        results = json.loads(res.content)
        places.extend(results['results'])
        time.sleep(2)
    return places

In [41]:
places = searchPlaces('starbucks', "40.819057,-73.914048", "500")

In [44]:
def getLocation(result):
    location = []
    for i in range(len(result)):
        latitude = result[i]['geometry']['location']['lat']
        longitude = result[i]['geometry']['location']['lng']
        name = result[i]['name']
        address = result[i]['formatted_address']
        loc = {
            'name': name,
            'address': address,
            'longitude': longitude,
            'latitude': latitude
        }
        location.append(loc)
    return location  

In [48]:
def facilities(coordinates):
    radius = '500'
    starbucks = getLocation(searchPlaces('starbucks', coordinates, radius))
    nightclubs = getLocation(searchPlaces('nightclubs', coordinates, radius))
    schools = getLocation(searchPlaces('elementary schools', coordinates, radius))
    vegan = getLocation(searchPlaces('vegan restaurants', coordinates, radius))
    starbucksdf = pd.DataFrame(starbucks)
    nightdf = pd.DataFrame(nightclubs)
    schooldf = pd.DataFrame(schools)
    vegandf = pd.DataFrame(vegan)
    df = pd.concat([starbucksdf,nightdf,schooldf,vegandf])
    return df

In [ ]:
primer = facilities('-122.323895,37.566879')

In [50]:
primer.shape()

NameError: name 'primer' is not defined

In [6]:
kinder_coords = [k['loc'] for k in kinder]
starb_coords = [s['loc'] for s in starbucks]
night_coords = [n['loc'] for n in nightclubs]
air_coords = [a['loc'] for a in airports]

In [7]:
mapp = folium.Map(location=[47.608013,-122.335167], zoom_start=12)

In [8]:
for k in kinder_coords:
    folium.Marker(k['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='pink'), 
                    ).add_to(mapp)
for s in starb_coords:
    folium.Marker(s['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='coffee-togo',color='green'), 
                    ).add_to(mapp)
for n in night_coords:
    folium.Marker(s['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cup',color='blue'), 
                    ).add_to(mapp)